# Baseline Models

In [1]:
import numpy as np

import gensim

import math

from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GRU, LSTM, Activation, Dense, Dropout, Embedding
from tensorflow.keras.callbacks import History 

D:\giuse\Conda\envs\thesis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
w2v_model = gensim.models.KeyedVectors.load("w2v.model", mmap='r')

In [3]:
vocab_size, emdedding_size = w2v_model.wv.vectors.shape
vocab_size, emdedding_size

(17862, 100)

In [4]:
x = np.load('data/x.npy')
y = np.load('data/y.npy')[:,0]

In [5]:
x.shape, y.shape

((5841, 100), (5841,))

In [6]:
# instantiate history to save losses
history = History()

## Baseline #1: GRU

In [25]:
gru = Sequential()

gru.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size))
gru.add(GRU(128, input_shape=(vocab_size, emdedding_size), return_sequences=True))
gru.add(Dropout(0.3))
gru.add(GRU(128))
gru.add(Dropout(0.3))
# gru.add(GRU(64))
# gru.add(Dropout(0.3))
gru.add(Dense(vocab_size, activation='softmax'))

In [26]:
gru.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [9]:
filepath = "weights/gru.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [27]:
gru_loss = gru.fit(x, y, validation_split=0.2, batch_size=64, epochs=10, callbacks=callbacks)

Train on 4672 samples, validate on 1169 samples
Epoch 1/10
4608/4672 [============================>.] - ETA: 0s - loss: 8.2565
Epoch 00001: val_loss did not improve from 7.43431
4672/4672 [==============================] - 38s 8ms/sample - loss: 8.2483 - val_loss: 7.5713
Epoch 2/10
4608/4672 [============================>.] - ETA: 0s - loss: 6.8660
Epoch 00002: val_loss did not improve from 7.43431
4672/4672 [==============================] - 34s 7ms/sample - loss: 6.8639 - val_loss: 7.6153
Epoch 3/10
4608/4672 [============================>.] - ETA: 0s - loss: 6.6458
Epoch 00003: val_loss did not improve from 7.43431
4672/4672 [==============================] - 34s 7ms/sample - loss: 6.6464 - val_loss: 7.5904
Epoch 4/10
4608/4672 [============================>.] - ETA: 0s - loss: 6.4402
Epoch 00004: val_loss did not improve from 7.43431
4672/4672 [==============================] - 33s 7ms/sample - loss: 6.4403 - val_loss: 7.5655
Epoch 5/10
4608/4672 [============================>.] - 

In [14]:
gru_loss = gru.fit(x, y, validation_split=0.2, batch_size=64, epochs=5, callbacks=callbacks)

Train on 4672 samples, validate on 1169 samples
Epoch 1/5
4608/4672 [============================>.] - ETA: 0s - loss: 8.2070
Epoch 00001: val_loss did not improve from 7.53704
4672/4672 [==============================] - 48s 10ms/sample - loss: 8.1900 - val_loss: 7.5887
Epoch 2/5
4608/4672 [============================>.] - ETA: 0s - loss: 6.8737
Epoch 00002: val_loss did not improve from 7.53704
4672/4672 [==============================] - 41s 9ms/sample - loss: 6.8751 - val_loss: 7.6231
Epoch 3/5
4608/4672 [============================>.] - ETA: 0s - loss: 6.6502
Epoch 00003: val_loss did not improve from 7.53704
4672/4672 [==============================] - 41s 9ms/sample - loss: 6.6537 - val_loss: 7.6106
Epoch 4/5
4608/4672 [============================>.] - ETA: 0s - loss: 6.4698
Epoch 00004: val_loss did not improve from 7.53704
4672/4672 [==============================] - 40s 9ms/sample - loss: 6.4722 - val_loss: 7.5743
Epoch 5/5
4608/4672 [============================>.] - ETA:

In [24]:
gru_loss = gru.fit(x, y, validation_split=0.2, batch_size=64, epochs=10, callbacks=callbacks)

Train on 4672 samples, validate on 1169 samples
Epoch 1/10
4608/4672 [============================>.] - ETA: 0s - loss: 8.5746
Epoch 00001: val_loss improved from 7.53704 to 7.43431, saving model to weights/gru.hdf5
4672/4672 [==============================] - 19s 4ms/sample - loss: 8.5571 - val_loss: 7.4343
Epoch 2/10
4608/4672 [============================>.] - ETA: 0s - loss: 6.8069
Epoch 00002: val_loss did not improve from 7.43431
4672/4672 [==============================] - 13s 3ms/sample - loss: 6.8050 - val_loss: 7.5220
Epoch 3/10
4608/4672 [============================>.] - ETA: 0s - loss: 6.6292
Epoch 00003: val_loss did not improve from 7.43431
4672/4672 [==============================] - 13s 3ms/sample - loss: 6.6298 - val_loss: 7.5766
Epoch 4/10
4608/4672 [============================>.] - ETA: 0s - loss: 6.5382
Epoch 00004: val_loss did not improve from 7.43431
4672/4672 [==============================] - 13s 3ms/sample - loss: 6.5377 - val_loss: 7.6314
Epoch 5/10
4608/46

## Baseline #2: GRU + Word2Vec

In [28]:
gru_w2v = Sequential()

gru_w2v.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[w2v_model.wv.vectors]))
gru_w2v.add(GRU(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
gru_w2v.add(Dropout(0.2))
gru_w2v.add(GRU(256, return_sequences=True))
gru_w2v.add(Dropout(0.2))
gru_w2v.add(GRU(128))
gru_w2v.add(Dropout(0.2))
gru_w2v.add(Dense(vocab_size, activation='softmax'))

In [29]:
gru_w2v.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [30]:
filepath = "weights/gru_w2v.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [31]:
gru_w2v_loss = gru_w2v.fit(x, y, validation_split=0.2, batch_size=64, epochs=20, callbacks=callbacks)

Train on 4672 samples, validate on 1169 samples
Epoch 1/20
4608/4672 [============================>.] - ETA: 1s - loss: 8.0941
Epoch 00001: val_loss improved from inf to 7.63390, saving model to weights/gru_w2v.hdf5
4672/4672 [==============================] - 116s 25ms/sample - loss: 8.0816 - val_loss: 7.6339
Epoch 2/20
4608/4672 [============================>.] - ETA: 1s - loss: 6.7905
Epoch 00002: val_loss did not improve from 7.63390
4672/4672 [==============================] - 113s 24ms/sample - loss: 6.7950 - val_loss: 7.6341
Epoch 3/20
4608/4672 [============================>.] - ETA: 1s - loss: 6.5816
Epoch 00003: val_loss did not improve from 7.63390
4672/4672 [==============================] - 113s 24ms/sample - loss: 6.5853 - val_loss: 7.6705
Epoch 4/20
4608/4672 [============================>.] - ETA: 1s - loss: 6.5015
Epoch 00004: val_loss did not improve from 7.63390
4672/4672 [==============================] - 113s 24ms/sample - loss: 6.5014 - val_loss: 7.7608
Epoch 5/20

## Baseline #3: LSTM 

In [32]:
lstm = Sequential()

lstm.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size))
lstm.add(GRU(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(GRU(256, return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(GRU(128))
lstm.add(Dropout(0.2))
lstm.add(Dense(vocab_size, activation='softmax'))

In [33]:
lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [34]:
filepath = "weights/lstm.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [35]:
lstm_loss = lstm.fit(x, y, validation_split=0.2, batch_size=64, epochs=10, callbacks=callbacks)

Train on 4672 samples, validate on 1169 samples
Epoch 1/20
4608/4672 [============================>.] - ETA: 1s - loss: 8.0984
Epoch 00001: val_loss improved from inf to 7.54207, saving model to weights/lstm.hdf5
4672/4672 [==============================] - 112s 24ms/sample - loss: 8.0884 - val_loss: 7.5421
Epoch 2/20
4608/4672 [============================>.] - ETA: 1s - loss: 6.7692
Epoch 00002: val_loss did not improve from 7.54207
4672/4672 [==============================] - 108s 23ms/sample - loss: 6.7673 - val_loss: 7.5893
Epoch 3/20
4608/4672 [============================>.] - ETA: 1s - loss: 6.5386
Epoch 00003: val_loss did not improve from 7.54207
4672/4672 [==============================] - 109s 23ms/sample - loss: 6.5299 - val_loss: 7.7323
Epoch 4/20
4608/4672 [============================>.] - ETA: 1s - loss: 6.3086
Epoch 00004: val_loss did not improve from 7.54207
4672/4672 [==============================] - 108s 23ms/sample - loss: 6.3078 - val_loss: 7.7210
Epoch 5/20
46

## Baseline #4: LSTM + Word2Vec 

In [36]:
lstm_w2v = Sequential()

lstm_w2v.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[w2v_model.wv.vectors]))
lstm_w2v.add(LSTM(256, input_shape=(vocab_size, emdedding_size), return_sequences=True))
lstm_w2v.add(Dropout(0.2))
lstm_w2v.add(LSTM(256, return_sequences=True))
lstm_w2v.add(Dropout(0.2))
lstm_w2v.add(LSTM(128))
lstm_w2v.add(Dropout(0.2))
lstm_w2v.add(Dense(vocab_size, activation='softmax'))

In [37]:
lstm_w2v.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [38]:
filepath = "weights/lstm_w2v.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [47]:
lstm_w2v_loss = lstm_w2v.fit(x, y, validation_split=0.2, batch_size=64, epochs=100, callbacks=callbacks)

Train on 4672 samples, validate on 1169 samples
Epoch 1/100
4608/4672 [============================>.] - ETA: 1s - loss: 7.0380
Epoch 00001: val_loss improved from inf to 7.51032, saving model to weights/lstm_w2v.hdf5
4672/4672 [==============================] - 103s 22ms/sample - loss: 7.0408 - val_loss: 7.5103
Epoch 2/100
4608/4672 [============================>.] - ETA: 1s - loss: 6.6845
Epoch 00002: val_loss did not improve from 7.51032
4672/4672 [==============================] - 110s 24ms/sample - loss: 6.6851 - val_loss: 7.5228
Epoch 3/100
4608/4672 [============================>.] - ETA: 1s - loss: 6.5174
Epoch 00003: val_loss did not improve from 7.51032
4672/4672 [==============================] - 114s 24ms/sample - loss: 6.5126 - val_loss: 7.5829
Epoch 4/100
4608/4672 [============================>.] - ETA: 1s - loss: 6.4291
Epoch 00004: val_loss did not improve from 7.51032
4672/4672 [==============================] - 156s 33ms/sample - loss: 6.4339 - val_loss: 7.5808
Epoch

## Generation

In [40]:
def sample(preds, temperature):
    if temperature <= 0:
        return np.argmax(preds)

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
  
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
  
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [41]:
def word_to_id(word):
    return w2v_model.wv.key_to_index[word]

def id_to_word(id):
    return w2v_model.wv.index_to_key[id]

In [42]:
def generate(model=gru, prompt='In this paper', words=20, temperature=0.2):
    word_ids = [word_to_id(word) for word in prompt.lower().split()]
    
    for i in range(words):
        prediction = model.predict(x=np.array(word_ids))
        id = sample(prediction[-1], temperature)
        word_ids.append(id)
    
    return ' '.join(id_to_word(id) for id in word_ids)

In [46]:
generate(model=gru_w2v, prompt='In this paper we present a novel approach')

'in this paper we present a novel approach faroese bots websites guards ideologies satisfies unibuc mance cialis automatizes royal utilit toute som bcp yifan entitled distorted approximation subtree'

## Evaluation

In [60]:
models = {gru_loss: 'GRU', gru_w2v_loss: 'GRU + Word2Vec', lstm_loss: 'LSTM', lstm_w2v_loss: 'LSTM + Word2Vec'}

In [66]:
# get minimimum validation loss for a model 
def min_val_loss(model, max_epochs=100):
    return min(model.history['val_loss'][:max_epochs])

In [70]:
for m in models.keys():
    print("Minimum validation loss for {}: {:.5f}".format(models[m], min_val_loss(m, 50)))
    print("Perplexity for model {}: {:.2f}\n".format(models[m], math.exp(min_val_loss(m))))

Minimum validation loss for GRU: 7.56549
Perplexity for model GRU: 1930.41

Minimum validation loss for GRU + Word2Vec: 5.64481
Perplexity for model GRU + Word2Vec: 282.82

Minimum validation loss for LSTM: 6.84608
Perplexity for model LSTM: 940.19

Minimum validation loss for LSTM + Word2Vec: 5.65056
Perplexity for model LSTM + Word2Vec: 234.66

